In [26]:
import clickhouse_connect
import pandas as pd
import numpy as np

In [34]:
client = clickhouse_connect.get_client(
    host='clickhouse-0-0.umetea.net',
    port=443,
    username='ml_ume',
    password='hDAoDvg8x552bH',
    database='dw',
    verify=False
)

customers_order = client.query_df("""
SELECT
    customer_id,
    count(DISTINCT order_id) AS order_cnt_12m,
    sum(total_amount) AS order_amt_12m,
    countIf(order_id, created_at_pt >= addMonths(now(), -6)) AS order_cnt_6m,
    sumIf(total_amount, created_at_pt >= addMonths(now(), -6)) AS order_amt_6m,
    min(created_at_pt) AS first_order_date,
    max(created_at_pt) AS last_order_date
FROM fact_orders
WHERE created_at_pt >= addMonths(now(), -12)
GROUP BY customer_id
""")

customers_lottery = client.query_df("""
SELECT
    customer_id,
    max(CASE WHEN joined=1 THEN 1 ELSE 0 END) AS lottery_joined,
    max(CASE WHEN redeemed_time_pt IS NOT NULL THEN 1 ELSE 0 END) AS lottery_redeemed
FROM dw.fact_lottery_histories
WHERE customer_created_at_pt >= addMonths(now(), -12)
GROUP BY customer_id
""")

promotion_past_12m = client.query_df("""
SELECT
    customer_id,
    count(DISTINCT order_id) AS coupon_order_cnt_12m
FROM dw.fact_orders
WHERE created_at_pt >= addMonths(now(), -12)
  AND (
      (campaign_names IS NOT NULL AND length(campaign_names) > 0)
      OR (source_type LIKE '%coupon%' OR source_type LIKE '%促销%')
      OR (discount > 0)
  )
GROUP BY customer_id

""")

category_preference = client.query_df("""
SELECT
    customer_id,
    arrayJoin([category_name]) AS major_category, -- 这里可进一步聚合求最多的品类
    count(*) AS category_order_cnt
FROM dw.fact_order_item_variations
WHERE created_at_pt >= addMonths(now(), -12)
GROUP BY customer_id, category_name
ORDER BY customer_id, category_order_cnt DESC
""")

client.close()


In [29]:
customers_order

,customer_id,order_cnt_12m,order_amt_12m,order_cnt_6m,order_amt_6m,first_order_date,last_order_date
0,DJCJCVCJBJPT5FKY7QXNZFRTJ8,2,25.84,2,25.84,2025-03-22 18:27:13,2025-04-18 15:08:30
1,H9NY66PM3GK4SY8BMEXKG5J3MC,2,58.07,2,58.07,2025-04-14 16:03:45,2025-04-23 15:27:49
2,3QRZTHATBAPQJ3PX17QVTRVR2M,1,17.18,0,None,2024-12-03 18:06:26,2024-12-03 18:06:26
3,KMV2KRQSF3FNP694G756959A58,1,21.40,1,21.40,2025-07-25 16:52:50,2025-07-25 16:52:50
4,6ZRFK7JVH4WR1H7QXYCVDNQ2N8,1,13.20,0,None,2024-09-27 15:06:36,2024-09-27 15:06:36
...,...,...,...,...,...,...,...
339653,9RH7K72WS16JVBX1SPZFF0TB30,2,26.29,2,26.29,2025-02-09 19:16:37,2025-04-14 15:08:50
339654,2109JDQ1A93JSFMDYM2WSRPBK4,4,87.94,2,23.58,2024-12-15 13:31:14,2025-07-19 14:33:30
339655,AHQ5PEXGCDE672Y7QC7KZBPAKC,1,7.80,1,7.80,2025-02-02 13:34:06,2025-02-02 13:34:06
339656,H2ZRXVG9KJ4ZQT8M5W52G6TBY8,1,13.00,0,None,2024-08-31 17:48:05,2024-08-31 17:48:05


In [35]:
promotion_past_12m

,customer_id,coupon_order_cnt_12m
0,<NA>,54666
1,GKDYM7ZM4TRW63AY2G56MRQWEM,1
2,61N6XQM3QX7719MT8XER3Z5GZC,1
3,XJJGP0XJ697G171MV51WEKGHZC,1
4,STKTTK2F4Z01WHCFZ2JQQ16NZM,1
...,...,...
39535,3VQJ5MRT19KCC6RKK29FMWECA8,2
39536,VGV5J51E3S49VD8JNY83JXEYX8,1
39537,HSE9JKAA3X4V18HZDRMS7P4TK4,1
39538,GC7ATT4PHVZYKC30EZF8YTWTEG,1


In [31]:
customers_lottery

,customer_id,lottery_joined,lottery_redeemed
0,1C91QQ3WB4AR8QX1F79W97MCVG,1,1
1,MF85TA6ADZJE5RMV0N9K3QWR70,1,1
2,2MDJ8E2N0C02BXZBRDWFD34W50,1,1
3,B8DCSWTSNS31K9PPXAPBGRJ2PC,1,1
4,FM1XWBW5957TD32X163ZN3Q5MG,1,1
...,...,...,...
2006,XVK5E2G88XB7A8V8967AX2J04M,1,1
2007,D9BKC698YX9YV83C2XQGE0KN1M,1,1
2008,67M27WPC2B69T49NFN4J4ZZV2C,1,1
2009,2HBHS3SPWF3AY5TDY6WWW75F9R,1,1


In [33]:
category_preference

,customer_id,major_category,category_order_cnt
0,0001VEK5ZX181768HPGV8M8V0G,Fruit Tea,2
1,0001VEK5ZX181768HPGV8M8V0G,Matcha,1
2,0001VEK5ZX181768HPGV8M8V0G,Caffeine-Free Drinks,1
3,0001VEK5ZX181768HPGV8M8V0G,Slushies,1
4,0001VEK5ZX181768HPGV8M8V0G,Milk Tea,1
...,...,...,...
673157,<NA>,Coconut,155
673158,<NA>,Milkshake,109
673159,<NA>,Diy Gelato,11
673160,<NA>,Fresh Tea(Lo),9
